# Bayesian Optimization

In [1]:
from bayes_opt import BayesianOptimization

# Bayesian Optimization 실습

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# 스케일링
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

# 인코딩
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis=1)
train = train.drop(columns = ['type'])

onehot = encoder.transform(test[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
test = pd.concat([test, onehot], axis=1)
test = test.drop(columns=['type'])

In [4]:
# 학습 데이터와 목표 변수 구분
x = train.drop(columns=['index', 'quality'])
y = train['quality']

# 랜덤포레스트 하이퍼파라미터 범위 지정
rf_parameter_bounds = {
    'max_depth': (1, 3), # 트리 깊이
    'n_estimators': (30, 100),
}

# 함수 생성
def rf_bo(max_depth, n_estimators):
    rf_params={
        'max_depth': int(round(max_depth)),
        'n_estimators': int(round(n_estimators)),
    }

    rf = RandomForestClassifier(**rf_params)

    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.2)

    rf.fit(x_train, y_train)
    score = accuracy_score(y_valid, rf.predict(x_valid))

    return score

In [5]:
# Bayesian Optimization 객체 생성
Bo_rf = BayesianOptimization(f=rf_bo, pbounds=rf_parameter_bounds, random_state=0)

In [6]:
# Bayesian Optimization 실행
Bo_rf.maximize(init_points=5, n_iter=5)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.53     |  2.098    |  80.06    |
|  2        |  0.5336   |  2.206    |  68.14    |
|  3        |  0.5218   |  1.847    |  75.21    |
|  4        |  0.5264   |  1.875    |  92.42    |
|  5        |  0.5327   |  2.927    |  56.84    |
|  6        |  0.4655   |  1.018    |  48.01    |
|  7        |  0.5382   |  2.596    |  62.47    |
|  8        |  0.5255   |  3.0      |  30.0     |
|  9        |  0.5491   |  3.0      |  100.0    |
|  10       |  0.4391   |  1.009    |  98.09    |


In [7]:
max_params = Bo_rf.max['params']

max_params['max_depth'] = int(max_params['max_depth'])
max_params['n_estimators'] = int(max_params['n_estimators'])
print(max_params)

{'max_depth': 3, 'n_estimators': 100}


In [8]:
# Bayesian Optimization 결과 저장
Bo_tuend_rf = RandomForestClassifier(**max_params)